<a href="https://colab.research.google.com/github/hsandaver/hsandaver/blob/main/ConservationPreprocessorV_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install necessary libraries
!pip install opencv-python-headless pillow ipywidgets matplotlib requests

# Import required libraries
from PIL import Image
import cv2
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, widgets
import requests
from io import BytesIO

# Step 1: Fetch image from IIIF server
# Replace this URL with the IIIF image URL you want to use
iiif_url = 'https://divinity.contentdm.oclc.org/iiif/2/p20081coll4:203/full/full/0/default.jpg'  # Example format, change to the actual IIIF image URL

# Fetch the image using the requests library
response = requests.get(iiif_url)
img = Image.open(BytesIO(response.content))

# Step 2: Convert image to grayscale
img_gray = img.convert('L')

# Step 3: Resize image while maintaining aspect ratio
max_size = 800
img_width, img_height = img_gray.size
aspect_ratio = img_width / img_height

if img_width > img_height:
    new_width = max_size
    new_height = int(max_size / aspect_ratio)
else:
    new_height = max_size
    new_width = int(max_size * aspect_ratio)

img_resized = img_gray.resize((new_width, new_height))
img_array = np.array(img_resized)

# Step 4: Interactive function for contrast and brightness adjustment
def adjust_image(clip_limit, grid_size):
    # Enhance contrast with CLAHE (Contrast Limited Adaptive Histogram Equalization)
    clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=(grid_size, grid_size))
    img_clahe = clahe.apply(img_array)

    # Display enhanced image with a larger size
    fig, ax = plt.subplots(1, 2, figsize=(14, 7), gridspec_kw={'width_ratios': [1, 1]})  # Increased figsize for larger output

    ax[0].imshow(img_array, cmap='gray')
    ax[0].set_title('Original Image')
    ax[0].axis('off')

    ax[1].imshow(img_clahe, cmap='gray')
    ax[1].set_title('Enhanced Image')
    ax[1].axis('off')

    plt.show()

# Step 5: Create interactive widgets
interact(adjust_image,
         clip_limit=widgets.FloatSlider(value=2.0, min=1.0, max=10.0, step=0.1, description='Clip Limit'),
         grid_size=widgets.IntSlider(value=8, min=1, max=16, step=1, description='Grid Size'));